In [2]:
import sqlite3
import pandas as pd
import numpy as np
import os
import requests

In [3]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('../data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table", error)

In [4]:
requete = "select code_bss from data_piezo"
liste_bss = [piezo[0] for piezo in requete_sql (requete)]

Connexion réussie à SQLite
Connexion SQLite est fermée


# Connexion à la bdd et création d'un dataframe

In [5]:
# Récupération des données 

connexion = sqlite3.connect('../data/liste_piezos.db')
curseur = connexion.cursor()
requete = """
    SELECT * FROM data_piezo    
    WHERE date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""
df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [6]:
df_piezo.head()

,date_debut,date_fin,nb_mesures_piezo,code_dpt,code_commune_insee,nom_commune,latitude,longitude,altitude_station,profondeur_investigation,...,noms_masse_eau_edl,fichier_piezo,fichier_era5,fichier_combined,commentaire,cluster_kmeans,FRANCE_lvl_1,FRANCE_lvl_2,FRANCE_lvl_3,FRANCE_lvl_4
code_bss,,,,,,,,,,,,,,,,,,,,,
03423X0056_100,1954-11-08,2022-05-30,3704,68,68113,Guémar,48.190195,7.396716,174.00,10.70,...,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène",1,1,1,None,GES_C,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3121
06216X0228_P30-10,2006-10-19,2022-05-03,5129,03,03204,Paray-sous-Briailles,46.272865,3.401730,237.20,8.00,...,Alluvions de l'Allier aval,1,1,1,None,ARA_A,FRANCE_3,FRANCE_31,FRANCE_312,FRANCE_3122
05068X0028_SP010,1990-12-11,2022-05-11,11386,85,85083,Épine,46.990356,-2.240129,2.25,16.50,...,Ile de Noirmoutier,1,1,1,None,None,None,None,None,None
09145X0057_N210,2002-01-09,2022-05-22,7248,84,84027,Caderousse,44.110203,4.751991,31.00,14.50,...,Alluvions des plaines du Comtat (Aigues Lez),1,0,0,None,PAC_A,FRANCE_3,FRANCE_31,FRANCE_313,FRANCE_3131
05483X0041_P210,2006-10-19,2011-03-17,1528,58,58225,Saincaize-Meauce,46.891482,3.067608,176.05,4.75,...,Alluvions de l'Allier aval,1,0,0,None,None,None,None,None,None


In [7]:
df_piezo.shape

(2740, 23)

# Fonction de collecte de la chronique d'un piezo

In [ ]:
def f_data_piezo (code_bss:str, date_index:bool=True):
    """
    code_bss : code bss brut
    date_index : (default: True), date en index du dataframe
    return : csv file
    """
    # Nom de fichier
    file  = f'{code_bss}.csv'
    if not os.path.isfile(f'../data/piezo/{file}') : #si le fichier n'existe pas
        code_bss_transfo = code_bss.replace("_", "%2F", 1) #Nécessaire pour construire l'URL
        url = f"http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss_transfo}&size=20000"
        print(url)
        try :
            get = requests.get(url)
            json = get.json()
            data = json['data']
            #Construction du dataframe
            df = pd.DataFrame(columns=['date_mesure', 'piezo'])
            df['date_mesure'] = pd.concat([pd.DataFrame([mesure['date_mesure']], columns=['date_mesure']) for mesure in data], ignore_index=True)
            df["piezo"] = pd.concat([pd.DataFrame([float(mesure['profondeur_nappe'])*(-1)], columns=['piezo']) for mesure in data],ignore_index=True)
            df.set_index('date_mesure', inplace=True)
            df.to_csv(f'../data/piezo/{file}', sep=";", encoding='utf-8', )
            print(f"Fichier {file} enregistré dans data/piezo.")
        except Exception as e:
            print(f"{code_bss} : Problème durant la requête à l'API, {e}.")
            if e == "No objects to concatenate" :
                return False
            
    else : # Si le fichier existe déjà on le mentionne.
        print(f"Le fichier {file} existe déjà.")

In [ ]:
# Test requete
url = f"http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=03423X0056%2F100&size=20000"
get = requests.get(url)
json = get.json()
data = json['data']
data

# Collecte de toutes les chroniques de la liste établie à partir de la bdd

In [ ]:
# Création d'une black liste des piezos sans données accessibles
piezo_black_list = []

# Liste des code_bss de la bdd
requete = """
        SELECT code_bss FROM data_piezo 
        WHERE date_debut < '2010-07-01'
        AND date_fin > '2010-07-01'
"""
liste_bss = [piezo[0] for piezo in requete_sql(requete)]

for piezo in liste_bss :
    if f_data_piezo(piezo) == False:
        # Renvoi le nom du piezo qui n'a pas de données
        piezo_black_list.append(piezo)
    
# Suppresion des piezos de la black list de la base de données
requete_del_black_list = f"""
                        DELETE FROM data_piezo
                        WHERE code_bss IN {tuple(piezo_black_list)}
                        """
requete_sql(requete_del_black_list)
print("chargement terminé")

# Mise à jour de la base de donnéees (champ fichier_piezo)
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")    
requete_fichier_piezo = f"""
        UPDATE data_piezo
        SET fichier_piezo =
        (CASE WHEN code_bss IN {tuple(bss_piezo)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_piezo)
print("Base de données à jour")